In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Parkinsons_Disease"
cohort = "GSE103099"

# Input paths
in_trait_dir = "../../input/GEO/Parkinsons_Disease"
in_cohort_dir = "../../input/GEO/Parkinsons_Disease/GSE103099"

# Output paths
out_data_file = "../../output/preprocess/Parkinsons_Disease/GSE103099.csv"
out_gene_data_file = "../../output/preprocess/Parkinsons_Disease/gene_data/GSE103099.csv"
out_clinical_data_file = "../../output/preprocess/Parkinsons_Disease/clinical_data/GSE103099.csv"
json_path = "../../output/preprocess/Parkinsons_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Helper-dependent CAV-2 vector-mediated LRRK2G2019S expression in the M. murinus brain induces Parkinson's disease-like histological lesions and motor symptoms"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['gender: female'], 1: ['age: 2 year old'], 2: ['cav-2 infection helper dependent 109 physical particles: no infection', 'cav-2 infection helper dependent 109 physical particles: Ipsilateral infection', 'cav-2 infection helper dependent 109 physical particles: Contralateral infection', 'cav-2 infection helper dependent 109 physical particles: Ipsilateral to CAV-2  infection', 'cav-2 infection helper dependent 109 physical particles: Contralateral to CAV-2 infection'], 3: ['brain tissue: Striatum', 'brain tissue: Frontal Cortex (FC)', 'brain tissue: Midbrain']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the output, we see that this dataset contains brain tissue samples
# This indicates it likely contains gene expression data, as mentioned in the series title
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait - Parkinson's Disease
# From the background information, this is a Parkinson's disease study with different conditions
# The key information appears to be in row 2, which shows CAV-2 infection status
trait_row = 2

# Age - Available in row 1
age_row = 1

# Gender - Available in row 0
gender_row = 0

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert the CAV-2 infection status to binary trait value for Parkinson's Disease"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Based on the data, we can infer that samples with "Ipsilateral infection" or "Ipsilateral to CAV-2 infection" 
    # are cases with Parkinson's Disease-like symptoms (as per the study title)
    if 'ipsilateral' in value.lower():
        return 1  # Case (PD model)
    elif 'no infection' in value.lower():
        return 0  # Control
    else:
        # Contralateral samples are likely controls as well
        return 0

def convert_age(value):
    """Convert age value to continuous format"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Extract numeric value
    import re
    numeric_match = re.search(r'(\d+(?:\.\d+)?)', value)
    if numeric_match:
        return float(numeric_match.group(1))
    return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    return None

# 3. Save Metadata - Initial filtering on usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Get clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("Preview of clinical features:")
    for col, values in preview.items():
        print(f"{col}: {values}")
    
    # Save the clinical features to the specified file
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of clinical features:
GSM2742123: [0.0, 2.0, 0.0]
GSM2742124: [0.0, 2.0, 0.0]
GSM2742125: [0.0, 2.0, 0.0]
GSM2742126: [0.0, 2.0, 0.0]
GSM2742127: [1.0, 2.0, 0.0]
GSM2742128: [1.0, 2.0, 0.0]
GSM2742129: [1.0, 2.0, 0.0]
GSM2742130: [1.0, 2.0, 0.0]
GSM2742131: [0.0, 2.0, 0.0]
GSM2742132: [0.0, 2.0, 0.0]
GSM2742133: [0.0, 2.0, 0.0]
GSM2742134: [0.0, 2.0, 0.0]
GSM2742135: [1.0, 2.0, 0.0]
GSM2742136: [1.0, 2.0, 0.0]
GSM2742137: [1.0, 2.0, 0.0]
GSM2742138: [1.0, 2.0, 0.0]
GSM2742139: [0.0, 2.0, 0.0]
GSM2742140: [0.0, 2.0, 0.0]
GSM2742141: [0.0, 2.0, 0.0]
GSM2742142: [0.0, 2.0, 0.0]
GSM2743405: [0.0, 2.0, 0.0]
GSM2743406: [0.0, 2.0, 0.0]
GSM2743407: [0.0, 2.0, 0.0]
GSM2743408: [0.0, 2.0, 0.0]
GSM2743409: [1.0, 2.0, 0.0]
GSM2743410: [1.0, 2.0, 0.0]
GSM2743411: [1.0, 2.0, 0.0]
GSM2743412: [1.0, 2.0, 0.0]
GSM2743413: [0.0, 2.0, 0.0]
GSM2743414: [0.0, 2.0, 0.0]
GSM2743415: [0.0, 2.0, 0.0]
GSM2743416: [0.0, 2.0, 0.0]
GSM2743417: [1.0, 2.0, 0.0]
GSM2743418: [1.0, 2.0, 0.0]
GSM2743419: [1.0, 

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers shown in the previous step

# These identifiers (like '1007_s_at', '1053_at', etc.) appear to be Affymetrix probe IDs
# rather than standard human gene symbols (which would look like BRCA1, TP53, etc.)

# Affymetrix probe IDs need to be mapped to standard gene symbols for analysis
# They follow the pattern of numbers followed by '_at', '_s_at', '_x_at', etc.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. First identify the columns in the gene annotation that contain probe IDs and gene symbols
# Based on the preview, 'ID' contains the probe IDs (e.g., '1007_s_at') and 'Gene Symbol' contains the gene symbols
prob_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Get the gene mapping dataframe using the get_gene_mapping function
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Check the first few rows of the mapped gene data to verify the transformation
print("Preview of gene-level expression data after mapping:")
print(gene_data.head())


Preview of gene-level expression data after mapping:
          GSM2742123  GSM2742124  GSM2742125  GSM2742126  GSM2742127  \
Gene                                                                   
A1BG       112.98580   51.284040    91.61227   57.792190    53.40241   
A1BG-AS1    35.40216   87.730980    12.45806   15.830780     9.63248   
A1CF       158.16758   15.254397   133.09628   46.997870    41.67348   
A2M        219.19014  146.995100   233.62709   98.652383   174.34974   
A2M-AS1     14.94806   73.829570    47.44039   67.683960    71.63633   

          GSM2742128  GSM2742129  GSM2742130  GSM2742131  GSM2742132  ...  \
Gene                                                                  ...   
A1BG        88.76160   76.598930   25.739190  105.839300   111.64980  ...   
A1BG-AS1    41.64716   16.129800   10.885150  155.726300    94.45621  ...   
A1CF       232.87153   31.342238  144.736461   43.862739   124.20535  ...   
A2M        176.68240  157.365300   91.370419  224.421200 

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols from the already mapped gene expression data from Step 6
# Apply normalization to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalization: {normalized_gene_data.shape}")
print("First 5 normalized gene symbols:")
print(normalized_gene_data.index[:5])

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data that was already processed in Step 2
# We need to load the clinical data from the file that was saved in Step 2
if os.path.exists(out_clinical_data_file):
    clinical_data_processed = pd.read_csv(out_clinical_data_file, index_col=0)
    print("Loaded clinical data from file")
else:
    # If for some reason the file wasn't saved, recreate the clinical features using the same parameters as in Step 2
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Use the same conversion function and trait_row from Step 2
    def convert_trait(value):
        """Convert occupation to binary trait (exposure risk for Parkinson's)"""
        if not isinstance(value, str):
            return None
        
        value = value.lower().split(": ")[-1].strip()
        
        if "farmworker" in value:
            return 1  # Higher pesticide exposure (risk factor for Parkinson's)
        elif "manual worker" in value:
            return 0  # Lower pesticide exposure 
        else:
            return None
    
    # Use the exact same parameters as we determined in Step 2
    clinical_data_processed = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=0,  # From Step 2
        convert_trait=convert_trait,
        age_row=None,  # From Step 2
        convert_age=None,
        gender_row=None,  # From Step 2
        convert_gender=None
    )
    
    # Save it again just to be sure
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_data_processed.to_csv(out_clinical_data_file)

print("Clinical data shape:", clinical_data_processed.shape)
print("Clinical data preview:")
print(preview_df(clinical_data_processed))

# 3. Link clinical and genetic data
# The clinical data is oriented with genes/traits as rows and samples as columns
# Transpose the normalized gene data to match this orientation (samples as columns)
genetic_data_t = normalized_gene_data

# Link clinical and genetic data vertically
linked_data = geo_link_clinical_genetic_data(clinical_data_processed, genetic_data_t)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Determine if trait and demographic features are biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Data shape after removing biased features: {linked_data.shape}")
print(f"Is the trait biased: {is_biased}")

# 6. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from blood samples comparing farmworkers (with higher pesticide exposure, a risk factor for Parkinson's) to manual workers."
)

# 7. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable. Linked data was not saved.")

Gene expression data shape after normalization: (19845, 59)
First 5 normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1'], dtype='object', name='Gene')


Normalized gene data saved to ../../output/preprocess/Parkinsons_Disease/gene_data/GSE103099.csv
Loaded clinical data from file
Clinical data shape: (3, 59)
Clinical data preview:
{'GSM2742123': [0.0, 2.0, 0.0], 'GSM2742124': [0.0, 2.0, 0.0], 'GSM2742125': [0.0, 2.0, 0.0], 'GSM2742126': [0.0, 2.0, 0.0], 'GSM2742127': [1.0, 2.0, 0.0], 'GSM2742128': [1.0, 2.0, 0.0], 'GSM2742129': [1.0, 2.0, 0.0], 'GSM2742130': [1.0, 2.0, 0.0], 'GSM2742131': [0.0, 2.0, 0.0], 'GSM2742132': [0.0, 2.0, 0.0], 'GSM2742133': [0.0, 2.0, 0.0], 'GSM2742134': [0.0, 2.0, 0.0], 'GSM2742135': [1.0, 2.0, 0.0], 'GSM2742136': [1.0, 2.0, 0.0], 'GSM2742137': [1.0, 2.0, 0.0], 'GSM2742138': [1.0, 2.0, 0.0], 'GSM2742139': [0.0, 2.0, 0.0], 'GSM2742140': [0.0, 2.0, 0.0], 'GSM2742141': [0.0, 2.0, 0.0], 'GSM2742142': [0.0, 2.0, 0.0], 'GSM2743405': [0.0, 2.0, 0.0], 'GSM2743406': [0.0, 2.0, 0.0], 'GSM2743407': [0.0, 2.0, 0.0], 'GSM2743408': [0.0, 2.0, 0.0], 'GSM2743409': [1.0, 2.0, 0.0], 'GSM2743410': [1.0, 2.0, 0.0], 'GSM2743411':

Data shape after handling missing values: (59, 19848)
For the feature 'Parkinsons_Disease', the least common label is '1.0' with 24 occurrences. This represents 40.68% of the dataset.
The distribution of the feature 'Parkinsons_Disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 2.0
  50% (Median): 2.0
  75%: 2.0
Min: 2.0
Max: 2.0
The distribution of the feature 'Age' in this dataset is severely biased.

For the feature 'Gender', the least common label is '0.0' with 59 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



Data shape after removing biased features: (59, 19846)
Is the trait biased: False


Linked data saved to ../../output/preprocess/Parkinsons_Disease/GSE103099.csv
